In [ ]:
import os
import tensorflow as tf #tf 2.0.0
import numpy as np
import pandas as pd
import pickle
from imutils import paths
import matplotlib.pyplot as plt
from tensorflow.keras import Model 
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
import random
from sklearn.preprocessing import StandardScaler # import the scaler
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import ConfusionMatrixDisplay
# Scikit learn for preprocessing
from sklearn.model_selection import train_test_split
import h5py
from tqdm import tqdm


# Image `Augmentation` for Increasing Dataset (Run Augmentation.py )

# ImageDataGenerator (in-place augmentation)
train_data_gen = ImageDataGenerator(rotation_range=50,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant',
                                    cval=0,
                                    rescale=1./255)
valid_data_gen = ImageDataGenerator(rotation_range=45,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='constant',
                                    cval=0,
                                    rescale=1./255)

test_data_gen = ImageDataGenerator(rescale=1./255)

dataset_dir = os.path.join('F:/GPU3/dataset11', "")

#dataset_dir = "E:/Progs/Restore/ManuscriptDataset/dataset11"

Batch_size =32
img_h = 512
img_w = 512
num_classes=4
n_epoch = 60

classes = ['crack', # 0
            'lackoffusion', # 1
            'porosity', # 2
            'slaginclusion', # 3
           ]

# Training
SEED = 42
#SEED = 1234
tf.random.set_seed(SEED) 

# initialize the data and labels
# x_train = []
# y_train = []
# x_test = []
# y_test = []


training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(512, 512),
                                               batch_size=Batch_size,
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors


# Validation
valid_dir = os.path.join(dataset_dir, 'testing')
valid_gen = valid_data_gen.flow_from_directory(valid_dir,
                                           target_size=(512, 512),
                                           batch_size=Batch_size, 
                                           classes=classes,
                                           class_mode='categorical',
                                           shuffle=False,
                                           seed=SEED)
# Test
test_dir = os.path.join(dataset_dir, 'testing')
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             target_size=(512, 512),
                                             batch_size=Batch_size,
                                             classes=classes,
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

                                
print('Train gen =',train_gen.n)
train_gen.reset()
x_train, y_train = next(train_gen)
for i in tqdm(range(int((train_gen.n)/Batch_size))): #1st batch is already fetched before the for loop.
  img, label = next(train_gen)
  x_train = np.append(x_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(x_train.shape, y_train.shape)

valid_gen.reset()
x_test, y_test = next(valid_gen)
for i in tqdm(range(int((valid_gen.n)/Batch_size))): #1st batch is already fetched before the for loop.
  img, label = next(valid_gen)
  x_test = np.append(x_test, img, axis=0 )
  y_test = np.append(y_test, label, axis=0)
print(x_test.shape, y_test.shape)

#print('Train size = ',x_train.shape," ",y_train.shape)
#print('Test size = ',x_test.shape," ",y_test.shape)
# #Visualization data
# CLASS_NAMES = np.array(['devanagari','nandinagiri', 'newari','sharda'], dtype='<U10')


# def show_batch(image_batch, label_batch):
#   plt.figure(figsize=(25,20))
#   for n in range(8):
#       ax = plt.subplot(1,8,n+1)
#       plt.imshow(image_batch[n])
#       plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
#       plt.axis('off')
      
# image_batch, label_batch = next(train_gen)
# show_batch(image_batch, label_batch)


# ResNet152V2 Model
ResNet_model = tf.keras.applications.ResNet152V2(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

# The last 15 layers fine tune
# for layer in ResNet_model.layers[:-15]:
#     layer.trainable = False

# x = ResNet_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dropout(0.5)(x)
# x = Flatten()(x)
# #x = Dense(units=2048, activation='relu')(x)
# #x = Dropout(0.25)(x)
# x = Dense(units=1024, activation='relu')(x)
# #x = Dropout(0.50)(x)
# output  = Dense(units=num_classes, activation='softmax')(x)
# model = Model(ResNet_model.input, output)

# The last 15 layers fine tune
for layer in ResNet_model.layers[:-15]:
    layer.trainable = False

x = ResNet_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(0.25)(x)
output  = Dense(units=num_classes, activation='softmax')(x)
model = Model(ResNet_model.input, output)
print("Successfully built the ResNet Model!")
print(model.summary())

loss = tf.keras.losses.CategoricalCrossentropy()
#loss = tf.keras.losses.KLDivergence()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=loss, metrics= ['accuracy'])
print("Model Compilation completed!")

mc = ModelCheckpoint('E:/Progs/Restore/best_modelresnet8.h5', monitor='val_accuracy', mode='max', save_best_only=True)
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

# lr_scheduler = LearningRateScheduler(lr_schedule)
#lrr = ReduceLROnPlateau(monitor='val_accuracy', 
                        # patience=3, 
                        # verbose=1, 
                        # factor=0.4, 
                        # min_lr=0.0001)


callbacks = [mc, es]

# model fit_generator
nb_train_samples = 1665
nb_validation_samples = 417

STEP_SIZE_TRAIN=nb_train_samples // Batch_size
STEP_SIZE_VALID=nb_validation_samples//Batch_size

# STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
# STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size

import time 
tstart = time.time()
transfer_learning_history = model.fit_generator(generator=train_gen,
                   steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=valid_gen,
                   validation_steps=STEP_SIZE_VALID,
                   epochs=n_epoch,
                   #callbacks=[mc,es]
                   callbacks=callbacks,
                  
                    
)

tend = time.time()
elapsed = (tend - tstart)/60
print("Model trained Successfully : Took - {} mins!".format(elapsed))

# model evaluate with validation set
scores = model.evaluate(valid_gen,verbose=0)

#print(transfer_learning_history.history.keys())

print("Test Loss Value: %.2f%%" % (scores[0]*100))
print("Test Accuracy Value:  %.2f%%"  % (scores[1]*100))

